# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [ ]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [ ]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

In [ ]:
links = fetch_website_links("https://www.nepremicnine.net")
links

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [ ]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [ ]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [ ]:
print(get_links_user_prompt("https://edwarddonner.com"))

In [ ]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [ ]:
select_relevant_links("https://edwarddonner.com")

In [ ]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [ ]:
select_relevant_links("https://edwarddonner.com")

In [ ]:
select_relevant_links("https://huggingface.co")

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [ ]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [ ]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

In [ ]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """


In [ ]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [ ]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

In [ ]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [ ]:
create_brochure("HuggingFace", "https://huggingface.co")

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [ ]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [ ]:
stream_brochure("HuggingFace", "https://huggingface.co")

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>

## My example of website fetching

In this example, I use website fetching together with multiple LLM calls (an early example of an **agentic AI design pattern**).  
I fetch recipe links from three different cooking websites and use an LLM to select only links that likely correspond to **individual requirements**.
From the selected recipes, I generate a **weekly meal-prep plan**.

In [ ]:
list_of_websides = ["https://www.bbcgoodfood.com/recipes/collection/healthy-recipes", "https://www.skinnytaste.com/recipe-index/", "https://www.allrecipes.com/recipes/84/healthy-recipes/"]

system_prompt_links = """
You are given a raw list of URLs collected from multiple cooking sites.
Your goal is to select URLs that are most likely to contain recipes that match the user's food preferences.
Only return URLs that look like individual recipe (not category pages, tag pages, search pages, author pages, homepages).
Prefer recipes that match required ingredients or at least have similar ingreadient.
Do not invent details. If preperation time is not obvious from the URL set estimated time to null.
If user forbids an ingredient for a meal type (e.g. "no banana for breakfast"), do not include breakfast recipes whose title indicates banana.
If a user indicates that it is allergic to a specific ingredient, do not include recipes whose title mentions that ingredient.
You should respond in JSON as in this example:
{
  "links": [
    {
      "url": string,
      "meal_type": "breakfast" | "snack" | "lunch" | "dinner",
      "matched_ingredients": string[],
      "estimated_time_min": number | null
    }
  ]
Return 30-60 links if possible. If fewer exist, return as many as you can.
"""

def get_user_prompt_links(url_list):
    required = ["banana", "chicken", "oats"]
    forbidden_by_meal = {"breakfast": ["banana"]}
    user_prompt = f"""
Required ingredients: {required}
Forbidden ingredients by meal type: {forbidden_by_meal}
I am allergic to rice.
Include only links that includes health recepies.
URLs:
"""
    for page in url_list:
        links = fetch_website_links(page)
        user_prompt += f"\n\n# Source page: {page}\n"
        user_prompt += "\n".join(links)
    return user_prompt

def get_relevant_links_of_different_websides(list_of_webs):
    response = openai.chat.completions.create(model = "gpt-4o-mini", 
    messages = [
        {"role": "system", "content": system_prompt_links},
        {"role": "user", "content":get_user_prompt_links(list_of_webs)}],
    response_format={"type": "json_object"})
    return json.loads(response.choices[0].message.content)

def fetch_pages_and_all_relevant_links(list_of_pages):
    relevant_links = get_relevant_links_of_different_websides(list_of_pages)
    result = "## Relevant Links:\n"
    for link in relevant_links.get("links", []):
        result += f"\n\n### {link.get('meal_type','unknown')} | {link.get('url','')}\n"
        result += fetch_website_contents(link["url"])
    return result



system_prompt_menu = """
You are a nutritionist and meal-prep planner.
You will receive a set of recipe texts.
Your task is to prepare a weekly MEAL PREP plan (Monday to Friday) for 1 person.
The goal is to cook most meals in advance (on Sunday) and portion them into boxes for the upcoming week.
Each day must include meals user asks for and the total calories per day must not exceed the daily calorie limit provided by the user.
All chosen recipes must fit within the preparation time requested by the user.
Prefer ingredient overlap across the week (reuse oats, chicken, vegetables, pantry items).
If calories or preparation time are not provided in the recipe, estimate them.
- In that case, clearly inform the user that the data was not available and that the values are estimated, using the wording:
  "Time/calories not available — estimated time/calories: ..."
Do not invent recipes, every recipe must be clearly grounded in the provided sources.
Include the source URL under each recipe title.
If a recipe repeats, reference the earlier occurrence instead of rewriting it, but do not repeat more than two recipes across the entire week.
You should respond in Markdown that: 
Start with:
"Hi, this is your weekly meal-prep plan, bon appétit!"
Then include:
1) A short description of the meal-prep strategy
2) Swap suggestions (protein / carb / fat alternatives), if the user asks
3) Daily overview (Monday-Friday):
   - meals for the day
   - total daily calories
4) Detailed recipes:
   - Ingredients (grams)
   - Steps
   - Time
   - Calories (provided or estimated)
   - Notes (storage tips)

Then create seperate section for sunday cooking plan.
Start with "Sunday Meal Prep Plan"
Then create section:
1) Sum ingredient quantities correctly. If for example a recipe uses 130 g of pork per serving and it is eaten on 5 days,
the total amount to cook is: 130 g x 5 = 650 g pork. Present this as a clear list of total ingredients to prepare.
2) Step by step cooking order
3) Storage recommendations
"""

def user_prompt_menu(list_of_websides):
    u_prompt_menu = """
Using the provided websites and recipes, prepare a weekly MEAL PREP plan.
Requirements:
Each day (Monday to Friday) must include:
  - breakfast
  - snack
  - lunch
  - dinner
The total calories per day must not exceed my daily calorie limit that is 1500kcal.
Only include recipes that take less than 40 minutes to prepare.
Prefer meals that can be cooked in advance and stored well.
Provide recommendations on how I can substitute the main sources of protein with alternatives.
Here are the recipes:
"""
    u_prompt_menu += fetch_pages_and_all_relevant_links(list_of_websides)
    return u_prompt_menu


   
def generate_weekly_menu(list_of_websides):
    response = openai.chat.completions.create(model="gpt-4o-mini", 
    messages = [
    {"role":"system", "content": system_prompt_menu},
    {"role":"user", "content": user_prompt_menu(list_of_websides)}]) 
    return display(Markdown(response.choices[0].message.content))

generate_weekly_menu(list_of_websides)

